# Selenium para descargar base de dato de la intranet de Opción de Energía

## Librerías

In [3]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
import pandas as pd
import glob
import os

## Page 1 - Login Opcion Comercializadora Intranet

In [176]:
driver = webdriver.Chrome("../Programas/chromedriver")

In [177]:
url = "https://canalagentes.opcionenergia.com/authentication/login"

In [178]:
driver.get(url)

In [179]:
driver.find_element_by_name("user").send_keys("vivolt")
driver.find_element_by_name("password").send_keys("temporal01vivolt")
driver.find_element_by_xpath("/html/body/app-root/app-agents/app-layout/app-login/section/div/div/div/div/form/div[2]/div[5]/div/div/div/button").send_keys(Keys.RETURN)

## Page 2 - Download "SIPS"

Para asignar la variable busco por el xpath, y desde tools de chrome, abro el codigo html y copio el xpath. 

Pulsando el boton de SIPS:

In [180]:
sips = driver.find_element_by_xpath('//*[@id="main_navbar"]/div[2]/div[1]/div[2]/ul[6]/li')

In [181]:
sips.click()

rellenando el CUPS: # agregar funcion para rellenar el cups desde la API

In [182]:
driver.find_element_by_name("Cups").send_keys("ES0021000005204736CV")

In [183]:
mostrar_grafico_sips = driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[1]/button')

In [184]:
mostrar_grafico_sips.click()

In [185]:
descargar_csv = driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[3]/button')

In [186]:
descargar_csv.click()

## Funciones para el proceso

In [17]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('../configuration')
from config import login_data, client_cups, url

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
#send keys con valores escondidos en config. Por ahora datos insertados de forma manual. 

In [19]:
def download_consumo(login_data, client_cups, url):
    driver = webdriver.Chrome("../Programas/chromedriver")
    driver.get(url)
    sleep(1.5)
    driver.find_element_by_name("user").send_keys("vivolt")
    driver.find_element_by_name("password").send_keys("temporal01vivolt")
    sleep(0.5)
    driver.find_element_by_xpath("/html/body/app-root/app-agents/app-layout/app-login/section/div/div/div/div/form/div[2]/div[5]/div/div/div/button").send_keys(Keys.RETURN)
    sleep(1.5)
    driver.find_element_by_xpath('//*[@id="main_navbar"]/div[2]/div[1]/div[2]/ul[6]/li').click()
    sleep(0.5)
    driver.find_element_by_name("Cups").send_keys("ES0021000005204736CV")
    sleep(0.5)
    driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[1]/button').click()
    sleep(0.5)
    driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[3]/button').click()

In [21]:
download_consumo(login_data, client_cups, url)

## Import CSV

In [7]:
def import_ConsumoCliente():
    list_of_files = glob.glob('/Users/pptrv/Downloads/*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)
    data = pd.read_csv(latest_file, sep=";;", error_bad_lines=False)
    return(data)

In [85]:
def import_ConsumoCliente():
    list_of_files = glob.glob('/Users/pptrv/Downloads/*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)
    data = pd.read_csv(latest_file, header=None, names=range(5))
    table_names = ["CUPS", "Tarifa", 'P1', 'P2', 'P3', 'P4', 'P5', 'P6','P1', 'P2', 'P3', 'P4', 'P5', 'P6']
    groups = data[0].isin(table_names).cumsum()
    tables = {g.iloc[0,0]: g.iloc[1:] for k,g in data.groupby(groups)}
    return(tables)

In [10]:
import_ConsumoCliente().head(15)

<ipython-input-7-74a5881e6523>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(latest_file, sep=";;", error_bad_lines=False)


,CUPS,ES0021000005204736CV
0,Tarifa,3.0A
1,Potencia Contratada (kWh),"P1;15,001"
2,Potencia Contratada (kWh),"P2;15,001"
3,Potencia Contratada (kWh),"P3;15,001"
4,Potencia Contratada (kWh),P4;0
5,Potencia Contratada (kWh),P5;0
6,Potencia Contratada (kWh),P6;0
7,Tension,3X400/230
8,Fecha ultima lectura,07/10/2020
9,Lecturas de activa(kWh):,None


## Cleaning DF

## Saving DF to CSV in Final_Proyect_Vivolt/Data/CSV

In [ ]:
consumo_cliente_df.to_csv("../Data/csv/bd_consumo_cliente.csv")